In [102]:
import seaborn as sns
import pandas as pd
import numpy as np

In [103]:
df = pd.read_json('data/data.json')
df.head()
df.describe()
df.columns

Index(['acct_type', 'approx_payout_date', 'body_length', 'channels', 'country',
       'currency', 'delivery_method', 'description', 'email_domain',
       'event_created', 'event_end', 'event_published', 'event_start',
       'fb_published', 'gts', 'has_analytics', 'has_header', 'has_logo',
       'listed', 'name', 'name_length', 'num_order', 'num_payouts',
       'object_id', 'org_desc', 'org_facebook', 'org_name', 'org_twitter',
       'payee_name', 'payout_type', 'previous_payouts', 'sale_duration',
       'sale_duration2', 'show_map', 'ticket_types', 'user_age',
       'user_created', 'user_type', 'venue_address', 'venue_country',
       'venue_latitude', 'venue_longitude', 'venue_name', 'venue_state'],
      dtype='object')

In [104]:
df['user_type'].unique()
df['acct_type'].unique()

array(['fraudster_event', 'premium', 'spammer_warn', 'fraudster',
       'spammer_limited', 'spammer_noinvite', 'locked', 'tos_lock',
       'tos_warn', 'fraudster_att', 'spammer_web', 'spammer'],
      dtype=object)

# Step 1: EDA

In [105]:
#1
df['fraud'] = df['acct_type'].apply(lambda x: 'fraud' in x)
df['fraud']

0         True
1        False
2        False
3        False
4        False
         ...  
14332     True
14333    False
14334    False
14335    False
14336     True
Name: fraud, Length: 14337, dtype: bool

In [106]:
df.drop(columns=['acct_type'], inplace=True)

In [107]:
#2
df[df['fraud']==True]['fraud'].count()
df[df['fraud']==False]['fraud'].count()
# Fraud = ~10%

13044

In [108]:
#3
features = ['body_length'    ,
'channels'       ,
'delivery_method',
'has_logo'       ,
'name_length'    ,
'org_facebook'   ,
'org_twitter'    ,
'sale_duration'  ,
'sale_duration2' ,
'user_age'       ,
'fraud']


In [109]:
df = df[features]

### Scoping the problem

1.) Preprocessing
- Convert dates to dt format
- Convert binary categories to boolean

2.) Models to Try

3.) Metric to use

# Step 2: Building the Model

In [110]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

df.dropna(inplace=True)


X = df.drop(columns=['fraud'])
y = df['fraud']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

In [128]:
X_train.count()
X_test.count()

body_length        3533
channels           3533
delivery_method    3533
has_logo           3533
name_length        3533
org_facebook       3533
org_twitter        3533
sale_duration      3533
sale_duration2     3533
user_age           3533
dtype: int64

In [143]:
from sklearn.metrics import confusion_matrix

def logistic_model(X, y):
    from sklearn.linear_model import LogisticRegression
    
    #select the columns with numerical/categorical data
    #X = X.loc[:, (X.dtypes == np.float64) or (X.dtypes == np.int64)]
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, stratify=y)
    
    
   
    
    #fit model
    model = LogisticRegression()
    model.fit(X_train, y_train)
    accuracy = model.score(X_valid, y_valid)
    coeffs = pd.DataFrame(model.coef_, columns=X_train.columns)
    print(coeffs)
    tn, fp, fn, tp = confusion_matrix(model.predict(X_valid), y_valid).ravel()
    
    return model, accuracy, (tn, fp, fn, tp)


def SGD_model(X, y):
    from sklearn.linear_model import SGDClassifier
    
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, stratify=y)
    
    #fit model
    model = SGDClassifier()
    model.fit(X_train, y_train)
    accuracy = model.score(X_valid, y_valid)
    tn, fp, fn, tp = confusion_matrix(model.predict(X_valid), y_valid).ravel()
  
    
    return model, accuracy, (tn, fp, fn, tp)
    
def rf_model(X,y):
    from sklearn.ensemble import RandomForestClassifier
    
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, stratify=y)
    
    #fit model
    model = RandomForestClassifier()
    model.fit(X_train, y_train)
    accuracy = model.score(X_valid, y_valid)
    tn, fp, fn, tp = confusion_matrix(model.predict(X_valid), y_valid).ravel()
    
    
    return model, accuracy, (tn, fp, fn, tp)

def gradient_model(X,y):
    from sklearn.ensemble import GradientBoostingClassifier
    
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, stratify=y)
    
    #fit model
    model = GradientBoostingClassifier()
    model.fit(X_train, y_train)
    accuracy = model.score(X_valid, y_valid)
    tn, fp, fn, tp = confusion_matrix(model.predict(X_valid), y_valid).ravel()
    
    
    return model, accuracy, (tn, fp, fn, tp)

In [141]:
logistic_model(X_train, y_train)

   body_length  channels  delivery_method  has_logo  name_length  \
0     -0.00006 -0.028705        -0.081677 -0.019338    -0.004285   

   org_facebook  org_twitter  sale_duration  sale_duration2  user_age  
0     -0.072061    -0.162572       0.030174       -0.056563 -0.003011  


/Users/jacobmullins/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


(LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=100,
                    multi_class='auto', n_jobs=None, penalty='l2',
                    random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                    warm_start=False),
 0.92,
 (2438, 212, 0, 0))

In [144]:
SGD_model(X_train, y_train)

(SGDClassifier(alpha=0.0001, average=False, class_weight=None,
               early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
               l1_ratio=0.15, learning_rate='optimal', loss='hinge',
               max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
               power_t=0.5, random_state=None, shuffle=True, tol=0.001,
               validation_fraction=0.1, verbose=0, warm_start=False),
 0.9230188679245283,
 (2349, 115, 89, 97))

In [145]:
gradient_model(X_train, y_train)

(GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                            learning_rate=0.1, loss='deviance', max_depth=3,
                            max_features=None, max_leaf_nodes=None,
                            min_impurity_decrease=0.0, min_impurity_split=None,
                            min_samples_leaf=1, min_samples_split=2,
                            min_weight_fraction_leaf=0.0, n_estimators=100,
                            n_iter_no_change=None, presort='deprecated',
                            random_state=None, subsample=1.0, tol=0.0001,
                            validation_fraction=0.1, verbose=0,
                            warm_start=False),
 0.969811320754717,
 (2409, 51, 29, 161))

In [146]:
rf_model(X_train, y_train)

(RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                        criterion='gini', max_depth=None, max_features='auto',
                        max_leaf_nodes=None, max_samples=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, n_estimators=100,
                        n_jobs=None, oob_score=False, random_state=None,
                        verbose=0, warm_start=False),
 0.9633962264150944,
 (2411, 70, 27, 142))